In [1]:
pip install pmdarima

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import warnings
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import json
from statsmodels.tsa.arima.model import ARIMA

warnings.filterwarnings('ignore')


# Function to predict commodity prices and save results as JSON
def predict(commodity, csv_path, json_path, pdq_low, pdq_high):
    print(f'Processing: {commodity}')
    
    try:
        # Read the CSV file and set 'Date' as the index
        df = pd.read_csv(csv_path, index_col='Date', parse_dates=True)
        
        # Find the last date in the dataset
        last_date = df.index[-1]
        
        # Set the future date range to be 31 days after the last date
        end_date = last_date + pd.Timedelta(days=31)
        index_future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), end=end_date)
        
        # Re-train the model and predict for 'High'
        model_high = ARIMA(df['High'], order=pdq_high).fit(method_kwargs={'warn_convergence': False})
        predict_high = model_high.predict(
            start=len(df), end=len(df) + len(index_future_dates) - 1, typ='levels'
        ).rename('High')
        
        # Re-train the model and predict for 'Low'
        model_low = ARIMA(df['Low'], order=pdq_low).fit(method_kwargs={'warn_convergence': False})
        predict_low = model_low.predict(
            start=len(df), end=len(df) + len(index_future_dates) - 1, typ='levels'
        ).rename('Low')
        
        # Combine results into a future DataFrame
        future_df = pd.DataFrame({
            'Date': index_future_dates,
            'High': predict_high.values,
            'Low': predict_low.values
        }).set_index('Date')
        future_df['Average'] = (future_df['High'] + future_df['Low']) / 2
        
        # Append future data to the original data
        combined_df = pd.concat([df, future_df]).reset_index()
        combined_df['Date'] = combined_df['Date'].dt.strftime('%Y-%m-%d')  # Format dates for JSON compatibility
        combined_df['Average'] = combined_df['Average'].fillna(0)  # Replace NaN with 0
        
        # Convert combined data to JSON format
        results = {
            'dates': combined_df['Date'].tolist(),
            'high_prices': combined_df['High'].tolist(),
            'low_prices': combined_df['Low'].tolist(),
            'average_prices': combined_df['Average'].tolist()
        }
        
        # Save results to a JSON file
        with open(json_path, 'w') as f:
            json.dump(results, f)
        
        return results
    
    except Exception as e:
        print(f"Error processing {commodity}: {e}")
        return {}


def process_commodities(commodities):
    # Base case: if only one group is left, process directly
    if len(commodities) == 1:
        group, specific_commodities = list(commodities.items())[0]
        results = []
        for specific_commodity, pdq_values in specific_commodities.items():
            for pdq in pdq_values:  # Iterate through the list of tuples
                # Unpack (p, d, q) for Low and High
                pdq_low = (pdq[0], pdq[1], pdq[2])  # Low
                pdq_high = (pdq[0], pdq[1], pdq[2])  # High (adjust if High has different parameters)
        
                csv_path = f'../csv/{group}/{specific_commodity}.csv'
                json_path = f'../json/{specific_commodity}.json'
                result = predict(specific_commodity, csv_path, json_path, pdq_low, pdq_high)
                results.append(result)
        return results

    # Divide step
    mid = len(commodities) // 2
    left_part = dict(list(commodities.items())[:mid])
    right_part = dict(list(commodities.items())[mid:])

    # Conquer: process each half recursively
    left_results = process_commodities(left_part)
    right_results = process_commodities(right_part)

    # Combine results
    return left_results + right_results


# Process commodity groups with concurrency
def process_all_groups(groups):
    results = []
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(process_commodities, {group: commodities})
            for group, commodities in groups.items()
        ]
        for future in futures:
            results.extend(future.result())
    return results


# Dictionary setup and loop to call the predict function
commodity_groups = {
    'rice': {
        'regular_milled_rice': [(10, 1, 28), (21, 1, 28)],  # Low, High
        'well_milled_rice': [(14, 1, 28), (3, 1, 28)],
        'premium_rice': [(7, 1, 28), (15, 1, 28)],
        'special_rice': [(3, 0, 28), (16, 1, 28)]
    },
    'meat': {
        'beef_brisket': [(14, 1, 28), (11, 0, 15)],
        'beef_rump': [(27, 1, 28), (24, 1, 28)],
        'whole_chicken': [(15, 1, 28), (3, 0, 28)],
        'pork_belly': [(13, 1, 28), (3, 1, 19)],
        'pork_kasim': [(11, 1, 28), (12, 1, 28)]
    },
    'fish': {
        'alumahan': [(11, 1, 28), (23, 1, 22)],
        'bangus': [(3, 1, 28), (2, 1, 28)],
        'galunggong': [(24, 1, 23), (13, 0, 25)],
        'tilapia': [(26, 0, 25), (2, 0, 26)]
    },
    'fruits': {
        'banana_lakatan': [(28, 1, 28), (21, 1, 28)],
        'calamansi': [(10, 1, 18), (15, 1, 21)],
        'mango': [(13, 1, 23), (9, 1, 28)],
        'papaya': [(18, 0, 28), (8, 0, 28)]
    },
    'vegetables': {
        'cabbage': [(17, 1, 24), (24, 1, 28)],
        'carrots': [(17, 1, 23), (4, 1, 24)],
        'eggplant': [(9, 0, 10), (8, 0, 13)],
        'tomato': [(25, 1, 20), (13, 1, 21)],
        'white_potato': [(4, 1, 28), (6, 1, 28)]
    },
    'spices': {
        'garlic': [(3, 1, 26), (4, 1, 28)],
        'red_onion': [(15, 0, 22), (11, 0, 18)]
    }
}

# Process all groups
results = process_all_groups(commodity_groups)
print("Processing completed.")

You have chosen beef_rump.


D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
D:\Programs\Py

{'dates': ['2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06', '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12', '2023-01-13', '2023-01-16', '2023-01-17', '2023-01-18', '2023-01-19', '2023-01-20', '2023-01-23', '2023-01-24', '2023-01-25', '2023-01-26', '2023-01-27', '2023-01-30', '2023-01-31', '2023-02-01', '2023-02-02', '2023-02-03', '2023-02-06', '2023-02-07', '2023-02-08', '2023-02-09', '2023-02-10', '2023-02-13', '2023-02-14', '2023-02-15', '2023-02-16', '2023-02-17', '2023-02-20', '2023-02-21', '2023-02-22', '2023-02-23', '2023-02-24', '2023-02-27', '2023-02-28', '2023-03-01', '2023-03-02', '2023-03-03', '2023-03-06', '2023-03-07', '2023-03-08', '2023-03-09', '2023-03-10', '2023-03-13', '2023-03-14', '2023-03-15', '2023-03-16', '2023-03-17', '2023-03-20', '2023-03-21', '2023-03-22', '2023-03-23', '2023-03-24', '2023-03-27', '2023-03-28', '2023-03-29', '2023-03-30', '2023-03-31', '2023-04-03', '2023-04-04', '2023-04-05', '2023-04-06', '2023-04-07', '2023-04-

D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['typ']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates

{'dates': ['2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06', '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12', '2023-01-13', '2023-01-16', '2023-01-17', '2023-01-18', '2023-01-19', '2023-01-20', '2023-01-23', '2023-01-24', '2023-01-25', '2023-01-26', '2023-01-27', '2023-01-30', '2023-01-31', '2023-02-01', '2023-02-02', '2023-02-03', '2023-02-06', '2023-02-07', '2023-02-08', '2023-02-09', '2023-02-10', '2023-02-13', '2023-02-14', '2023-02-15', '2023-02-16', '2023-02-17', '2023-02-20', '2023-02-21', '2023-02-22', '2023-02-23', '2023-02-24', '2023-02-27', '2023-02-28', '2023-03-01', '2023-03-02', '2023-03-03', '2023-03-06', '2023-03-07', '2023-03-08', '2023-03-09', '2023-03-10', '2023-03-13', '2023-03-14', '2023-03-15', '2023-03-16', '2023-03-17', '2023-03-20', '2023-03-21', '2023-03-22', '2023-03-23', '2023-03-24', '2023-03-27', '2023-03-28', '2023-03-29', '2023-03-30', '2023-03-31', '2023-04-03', '2023-04-04', '2023-04-05', '2023-04-06', '2023-04-07', '2023-04-

D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['typ']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates

{'dates': ['2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06', '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12', '2023-01-13', '2023-01-16', '2023-01-17', '2023-01-18', '2023-01-19', '2023-01-20', '2023-01-23', '2023-01-24', '2023-01-25', '2023-01-26', '2023-01-27', '2023-01-30', '2023-01-31', '2023-02-01', '2023-02-02', '2023-02-03', '2023-02-06', '2023-02-07', '2023-02-08', '2023-02-09', '2023-02-10', '2023-02-13', '2023-02-14', '2023-02-15', '2023-02-16', '2023-02-17', '2023-02-20', '2023-02-21', '2023-02-22', '2023-02-23', '2023-02-24', '2023-02-27', '2023-02-28', '2023-03-01', '2023-03-02', '2023-03-03', '2023-03-06', '2023-03-07', '2023-03-08', '2023-03-09', '2023-03-10', '2023-03-13', '2023-03-14', '2023-03-15', '2023-03-16', '2023-03-17', '2023-03-20', '2023-03-21', '2023-03-22', '2023-03-23', '2023-03-24', '2023-03-27', '2023-03-28', '2023-03-29', '2023-03-30', '2023-03-31', '2023-04-03', '2023-04-04', '2023-04-05', '2023-04-06', '2023-04-07', '2023-04-

D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['typ']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
